In [1]:
import os
import numpy as np
import pandas as pd
import deepchem as dc
import tempfile
import os
import nglview
import locale

from deepchem.utils import download_url, load_from_disk
from deepchem.utils.vina_utils import prepare_inputs
from openmm.app import PDBFile
from pdbfixer import PDBFixer
from rdkit import Chem
from rdkit.Chem import AllChem

/tmp/ipykernel_3095/126294797.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading some PyTorch models, missing a dependency. No module named 'torch'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'torch'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
data_dir = dc.utils.get_data_dir()
dataset_file = os.path.join(data_dir, "pdbbind_core_df.csv.gz")

if not os.path.exists(dataset_file):
    print('File does not exist. Downloading file...')
    download_url("https://s3-us-west-1.amazonaws.com/deepchem.io/datasets/pdbbind_core_df.csv.gz")
    print('File downloaded...')

raw_dataset = load_from_disk(dataset_file)
raw_dataset = raw_dataset[['pdb_id', 'smiles', 'label']]

In [3]:
raw_dataset

,pdb_id,smiles,label
0,2d3u,CC1CCCCC1S(O)(O)NC1CC(C2CCC(CN)CC2)SC1C(O)O,6.92
1,3cyx,CC(C)(C)NC(O)C1CC2CCCCC2C[NH+]1CC(O)C(CC1CCCCC...,8.00
2,3uo4,OC(O)C1CCC(NC2NCCC(NC3CCCCC3C3CCCCC3)N2)CC1,6.52
3,1p1q,CC1ONC(O)C1CC([NH3+])C(O)O,4.89
4,3ag9,NC(O)C(CCC[NH2+]C([NH3+])[NH3+])NC(O)C(CCC[NH2...,8.05
...,...,...,...
188,2x0y,CN1C2NCN(CC(O)CO)C2C(O)N(C)C1O,4.60
189,3uex,CCCCCCCCCCCCCCCCCC(O)O,6.92
190,2pq9,[O-]C(O)C1CC(OC(O[PH](O)(O)O)(C([O-])O)C(F)F)C...,8.11
191,1u1b,CC1CN(C2CC(O[PH](O)(O)O[PH](O)(O)OCC3OC(N4CNC5...,7.80


In [4]:
ligands10 = raw_dataset['smiles'].iloc[0:10]
generated_pdb = 'data/angptl7.pdb'
generated_pdb_no_extension = os.path.splitext(os.path.basename(generated_pdb))[0]

In [5]:
count=0
scores_matrix =[]
complex_mol_array = []

In [6]:
%%time

for count in range(0, 1):
    
  print("Docking ligand "+str(count))
  ligand = ligands10[count]
  p, m = None, None

  vpg = dc.dock.pose_generation.VinaPoseGenerator(
      pocket_finder=dc.dock.binding_pocket.ConvexHullPocketFinder())

  try:
      p, m = prepare_inputs('%s' % (generated_pdb), ligand)
  except:
      print('%s failed PDB fixing' % (generated_pdb))

  if p and m:  # protein and molecule are readable by RDKit
      print(generated_pdb, p.GetNumAtoms())
      Chem.rdmolfiles.MolToPDBFile(p, 'data/protein_%s.pdb' % (count))
      Chem.rdmolfiles.MolToPDBFile(m, 'data/ligand_%s.pdb' % (count))

  complexes, scores = vpg.generate_poses(
      molecular_complex=('data/protein_%s.pdb' % (count),'data/ligand_%s.pdb' % (count)), 
      out_dir='data',generate_scores=True, num_pockets=3)
    
  complex_mol = Chem.CombineMols(complexes[0][0], complexes[0][1])
  complex_mol_array.append(complex_mol)
  print(scores)
  scores_matrix.append(scores)

<timed exec>:11: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.


Docking ligand 0


[16:30:57] UFFTYPER: Unrecognized atom type: S_5+4 (7)


data/angptl7.pdb 2815
Computing Vina grid ... done.
Performing docking (random seed: -1806717614) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************



mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -5.145          0          0
   2       -4.952      3.487      7.944
   3       -4.907      1.626      2.536
   4       -4.703      2.778      4.374
   5       -4.681      4.825      8.374
   6       -4.679      2.135      3.862
   7       -4.505      3.217      8.068
   8       -2.349      3.283      7.626
   9        4.614      2.913      8.412
Computing Vina grid ... done.
Performing docking (random seed: -1806717614) ... 


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -2.542          0          0
   2       -1.459      1.985      3.059
   3      -0.9452      3.412      6.686
   4       -0.363      3.815      5.244
   5       0.1563      2.687      4.155
   6        2.036      3.521       4.48
   7        10.07      2.234      2.928
   8        38.89      3.659      5.756
   9        128.5      2.276      4.121
Computing Vina grid ... done.
Performing docking (random seed: -1806717614) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
[-5.145, -4.952, -4.907, -4.703, -4.681, -4.679, -4.505, -2.349, -2.542, -1.459, -0.945, -0.363, 0.156, -4.241, -4.04

In [7]:
scores_matrix

[[-5.145,
  -4.952,
  -4.907,
  -4.703,
  -4.681,
  -4.679,
  -4.505,
  -2.349,
  -2.542,
  -1.459,
  -0.945,
  -0.363,
  0.156,
  -4.241,
  -4.046,
  -4.044,
  -3.985,
  -3.961,
  -3.883,
  -3.839,
  -3.786,
  -3.783]]

In [ ]:
import nglview


In [ ]:
v = nglview.show_rdkit(complex_mol_array[2])
display(v)